# Exploratory Data Analysis

In [ ]:
import pandas as pd
import torch.optim as optim
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler
from windrose import WindroseAxes

In [ ]:
df = pd.read_csv("Proj_NWP_Case3.csv")
df2 = pd.read_csv("Proj_Measurements_Case3.csv")
df.Date_Time=pd.to_datetime(df.Date_Time)
df = df.set_index('Date_Time')
df2.Date_Time=pd.to_datetime(df2.Date_Time)
df2 = df2.set_index('Date_Time')

In [ ]:
#Merge the two datasets
df_merged = df.merge(df2['Park Power [KW]'], how='left', left_index=True, right_index=True)

print(df_merged.isnull().sum(axis = 0))

#Remove Nan-values
df_merged = df_merged.dropna()

In [ ]:
df_merged.info()

In [ ]:
df_merged.shape

In [ ]:
df_merged.describe()

### Check for outliers

In [ ]:
l = df_merged.columns.values
number_of_columns=18
number_of_rows = len(l)-1/number_of_columns
plt.figure(figsize=(number_of_columns,5*number_of_rows))
for i in range(0,len(l)):
    plt.subplot(number_of_rows + 1,number_of_columns,i+1)
    sns.set_style('whitegrid')
    sns.boxplot(df_merged[l[i]],color='green',orient='vertical')
    plt.tight_layout()

It looks like there are outliers in 7 of the 18 features. These are speed_10m, speed_50m, Direction_100m, Speed_100m, Direction_150m, Speed_150m and Park Power \[KW\]. Will further look into Park Power \[KW\], because this is our target variable.

In [ ]:
sns.boxplot(data=df_merged['Park Power [KW]'])
plt.savefig('boxsplot_PP.png')

### Checking the distribution-Skewness

In [ ]:
plt.figure(figsize=(5*number_of_columns,5*number_of_rows))
for i in range(0,len(l)):
    plt.subplot(number_of_rows * 2,4*number_of_columns,i+1)
    sns.distplot(df_merged[l[i]],kde=True) 

### Correlation plot

In [ ]:
corr = df_merged.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.savefig('Corr_plot.png')

Park Power \[KW\] have the biggest correlation with the wind speed, but it doesn't look like a problem. The temerature of all heights have a high negative correlation with the air density of all heights.

### Wind speed

In [ ]:
direction = ["Direction_10m","Direction_50m","Direction_100m","Direction_150m"]
speed = ["Speed_10m","Speed_50m","Speed_100m","Speed_150m"]
titles = ["10m","50m","100m","150m"]
ax = WindroseAxes.from_ax()
ax.bar(df_merged[direction[2]], df_merged[speed[2]], normed=True, opening=1)
ax.set_legend(units="m/s")
ax.set_xticklabels(['E', 'NE', 'N', 'NW',  'W', 'SW', 'S', 'SE'])
ax.set_title("Windrose for " + titles[2])
plt.savefig('100m_wind.png')

In [ ]:
direction = ["Direction_10m","Direction_50m","Direction_100m","Direction_150m"]
speed = ["Speed_10m","Speed_50m","Speed_100m","Speed_150m"]
temperature = ["Temperature_10m","Temperature_50m","Temperature_100m","Temperature_150m"]
density = ["Air Density_10m","Air Density_50m","Air Density_100m","Air Density_150m"]
titles = ["10m","50m","100m","150m"]

for i in range(len(speed)):
    ax = WindroseAxes.from_ax()
    ax.bar(df_merged[direction[i]], df_merged[speed[i]], normed=True, opening=1)
    ax.set_legend(units="m/s")
    ax.set_xticklabels(['E', 'NE', 'N', 'NW',  'W', 'SW', 'S', 'SE'])
    ax.set_title("Windrose for " + titles[i])

The wind direction is mostly from the south-west

### Seasonality

In [ ]:
df_merged['Month'] = df_merged.index.month
ylabels=["[m/s]","Degrees","Degree celsius"]
fig, axes = plt.subplots(4, 1, figsize=(11, 10), sharex=True)
for name,label, ax in zip(['Speed_10m', 'Direction_10m', 'Temperature_10m',"Air Density_10m"],["[m/s]","Degree (Angle)","Degree celsius","ρ"], axes):
    sns.boxplot(data=df_merged, x='Month', y=name, ax=ax)
    ax.set_ylabel(label)
    ax.set_title(name)
    if ax != axes[-1]:
        ax.set_xlabel('')

### Baseline-machine learning (Linear Regression)

In [ ]:
df_merged_model=df_merged.copy()

#Data Standardization
df_merged_model_mean = df_merged_model.mean(axis=0)
df_merged_model_std = df_merged_model.std(axis=0)
df_merged_model_sd= (df_merged_model - df_merged_model_mean) / df_merged_model_std

#Splitting the data
split=int(len(df_merged_model_sd)*7/10)
training=df_merged_model_sd[:split]
test=df_merged_model_sd[split:]


#Making test and train sets
x_cols=['Direction_10m', 'Speed_10m', 'Temperature_10m', 'Pressure_seaLevel',
       'Air Density_10m', 'Direction_50m', 'Speed_50m', 'Temperature_50m',
       'Air Density_50m', 'Direction_100m', 'Speed_100m', 'Temperature_100m',
       'Air Density_100m', 'Direction_150m', 'Speed_150m', 'Temperature_150m',
       'Air Density_150m']

x_train=training[x_cols]
x_test=test[x_cols]

y_train=training['Park Power [KW]']
y_test=test['Park Power [KW]']

#Train the model
regr=linear_model.LinearRegression(fit_intercept=False)
regr.fit(x_train, y_train);
print(regr.coef_)

#Test the model
scorelr=regr.score(x_test, y_test)
print(f' The accuracy is {scorelr}')

## Feature Engineering

In [ ]:
def cos_sin_time(df):
    #Extracting the minute of the hour, hour of day and day of the month
    df["hour"] = [x.hour for x in df["dt"]]
    df["Month"] = [x.month for x in df["dt"]]
    df["Year"] = [x.year for x in df["dt"]]
    df["timestamp"] = [x.timestamp() for x in df["dt"]]
    
    
    
    ## Creating the cyclical feature 
    
    # day
    df["day_cos"] = [np.cos(x * (2 * np.pi / 23)) for x in df["hour"]]
    df["day_sin"] = [np.sin(x * (2 * np.pi / 23)) for x in df["hour"]]
    
    # Seconds in day 
    s = 24 * 60 * 60
    # Seconds in year 
    year = (365.25) * s
    df["month_cos"] = [np.cos(x * (2 * np.pi / year)) for x in df["timestamp"]]
    df["month_sin"] = [np.sin(x * (2 * np.pi / year)) for x in df["timestamp"]]
    
def change_in_direction(df):

    # 10 Meter
    df['Change_Speed_10'] = df['Speed_10m'].diff()
    df['Change_Speed_10'] = df['Change_Speed_10'].fillna(0)
    df['Change_direction_10'] = df['Direction_10m'].diff()
    df['Change_direction_10'] = df['Change_direction_10'].fillna(0)

    # 50 Meter
    df['Change_Speed_50'] = df['Speed_50m'].diff()
    df['Change_Speed_50'] = df['Change_Speed_50'].fillna(0)
    df['Change_direction_50'] = df['Direction_50m'].diff()
    df['Change_direction_50'] = df['Change_direction_50'].fillna(0)

    # 100 Meter
    df['Change_Speed_100'] = df['Speed_100m'].diff()
    df['Change_Speed_100'] = df['Change_Speed_100'].fillna(0)
    df['Change_direction_100'] = df['Direction_100m'].diff()
    df['Change_direction_100'] = df['Change_direction_100'].fillna(0)

    # 150 Meter
    df['Change_Speed_150'] = df['Speed_150m'].diff()
    df['Change_Speed_150'] = df['Change_Speed_150'].fillna(0)
    df['Change_direction_150'] = df['Direction_150m'].diff()
    df['Change_direction_150'] = df['Change_direction_150'].fillna(0)

In [ ]:
cut_off=df['Park Power [KW]'].quantile(0.9)

#Setting the max theoretical power output
Capacity = 33*1500
df=df.dropna()

for i in range(1,len(df)):
    if (df['Park Power [KW]'][i] >= Capacity) & (df['Park Power [KW]'][i-1] > cut_off) & (df['Change_Speed_50'][i] > 0.1):
        df['Park Power [KW]'][i]=Capacity
        
df_cleaned=df[(df['Park Power [KW]']<=Capacity) & (df['Park Power [KW]']>0)].dropna()

## feature engineering

In [ ]:
# Making wind vectors

wd_rad_10 = df_cleaned.pop('Direction_10m')*np.pi / 180
wd_rad_50 = df_cleaned.pop('Direction_50m')*np.pi / 180
wd_rad_100 = df_cleaned.pop('Direction_100m')*np.pi / 180
wd_rad_150 = df_cleaned.pop('Direction_150m')*np.pi / 180

ws_10 = df_cleaned.pop('Speed_10m')
ws_50 = df_cleaned.pop('Speed_50m')
ws_100 = df_cleaned.pop('Speed_100m')
ws_150 = df_cleaned.pop('Speed_150m')

# Calculate the wind x and y components.
df_cleaned['Wx_10'] = ws_10*np.cos(wd_rad_10)
df_cleaned['Wy_10'] = ws_10*np.sin(wd_rad_10)
df_cleaned['Wx_50'] = ws_50*np.cos(wd_rad_50)
df_cleaned['Wy_50'] = ws_50*np.sin(wd_rad_50)
df_cleaned['Wx_100'] = ws_100*np.cos(wd_rad_100)
df_cleaned['Wy_100'] = ws_100*np.sin(wd_rad_100)
df_cleaned['Wx_150'] = ws_150*np.cos(wd_rad_150)
df_cleaned['Wy_150'] = ws_150*np.sin(wd_rad_150)

In [ ]:
df_cleaned=df_cleaned.drop(["dt","timestamp","hour","Month","Year","10m_cos","10m_sin","50m_cos","50m_sin","100m_sin","100m_cos","150m_sin","150m_cos"],axis=1)